The goal of scientific research is to advance the broader collection of knowledge about a topic.
But the path to socialize this research is full of messy, private detours.

The Camber [`stash`](/docs/python-api/stash) package offers an interface to pass your data, code, and analysis between private, team, and open files.
In this tutorial, follow along to learn how use stash to:
1. View your stashes.
1. Transfer data from open to private stashes.
1. Transfer data from the private to the open stash.


## View Stashes

First, import Camber and create variables for your stashes.


In [1]:
import camber
private = camber.stash.private_stash
team = camber.stash.team_stash["<MY_TEAM_NAME>"]
open = camber.stash.open_stash

Note that you can belong to multiple teams, so you must specify the specific team stash you want to work with.

The team and open stashes have datasets.
Inspect them with the [`ls()`](/docs/python-api/stash/#ls) method:

In [2]:
print("team stash data:", team.ls("datasets"),"\n",
      "open stash data:", open.ls("datasets"))

team stash data: ['calories/', 'kaggle/'] 
 open stash data: ['beginners/', 'esoreflex/', 'gaia/']


Now that you can list the contents of these stashes, it's time to share data between them.

## Copy from open to private stashes

Use  `ls()` to list the files in the open stash `beginners/` dataset:

In [19]:
open.ls("datasets/beginners")

['cereal.csv', 'titanic.csv']



Open stashes are read-only. To manipulate an open dataset, you need to copy it to your private stash.
Before doing that, though, make a file in your Jupyter space called `stash-tutorial`.
This is to help keep your private stash organized.

Now use the [`cp()`](/docs/python-api/stash/#cp) method to copy the [cereal dataset](https://www.kaggle.com/datasets/crawford/80-cereals) from the open stash to the `stash-tutorial/` directory in your private stash:


In [4]:
open.cp(
		dest_stash=private,
    	src_path="datasets/beginners/cereal.csv",
		dest_path="~/stash-tutorial/cereal.csv",
)


Confirm that it's in your private stash:


In [5]:
private.ls("~/stash-tutorial")

['cereal.csv', 'stash-tutorial.ipynb']



All there. Now, before starting analysis, make a file called `results.txt`.
This file might come in handy:


In [6]:
!touch ~/stash-tutorial/results.txt

Confirm it was created:

In [8]:
private.ls("~/stash-tutorial")

['cereal.csv', 'results.txt', 'stash-tutorial.ipynb']

In this private stash, you can safely do write operations on your files.

## Analyze your copy of the data

You want to determine which item from the dataset has the highest protein content relative to its number of calories.
To achieve this, you can use the [pandas]() package, which comes installed on the Camber kernal. 

First, import pandas and initialize the dataframe:

In [18]:
import pandas as pd
df = pd.read_csv('~/cereal-analysis/cereal.csv')

Then calculate the protein-to-calorie ratio:

In [10]:

# Calculate the ratio
df['calories_to_protein'] =  df['protein'] / df['calories'] 

# Find the cereal with the best ratio:
max_ratio_row = df.loc[df['calories_to_protein'].idxmax()]
highest_ratio_cereal = max_ratio_row['name']

print(f'The cereal with the best protein-to-calorie ratio is:\n{highest_ratio_cereal}')

The cereal with the best protein-to-calorie ratio is:
All-Bran with Extra Fiber


This is useful information―your team might want to know.

## Share your analysis with your team stash

Make a new CSV file to share with your team stash.

In [11]:
ratio_df = df[['name','calories','protein', 'calories_to_protein']]
ratio_df.to_csv('~/stash-tutorial/calories-to-protein.csv', index=False)
# ratio_df.head() Uncomment to inspect the file structure



Now copy the contents of your `stash-tutorial` directory to your team stash:



In [14]:
private.cp(
    src_path="~/stash-tutorial/",
    dest_stash=team,
    dest_path="~/datasets/calories/",
    recursive=True,
)

Check that the file is in the stash directory:

In [15]:
team.ls("~/datasets/calories")

['calories-to-protein.csv',
 'cereal.csv',
 'results.txt',
 'stash-tutorial.ipynb']



Oops! It turns out that you never used the `results.txt` file.
And, since the `cp()` method had the `recursive` flag, this empty file got copied over to your team stash.

No worries. Just remove it with the [`rm()`](/docs/python-api/stash/#rm) method:



In [13]:
team.rm("~/datasets/calories/results.txt")
team.ls("~/datasets/calories")

['calories-to-protein.csv', 'cereal.csv', 'stash-tutorial.ipynb']

Now tell your team about the interesting cereal analysis you just uploaded to their stash.